In [1]:
# %% 1. การติดตั้งและ Import Library

# ติดตั้ง library ที่จำเป็นจาก Hugging Face และ scikit-learn
!pip install -q transformers[torch] datasets accelerate scikit-learn

import pandas as pd
import numpy as np
import torch
from datasets import load_dataset, Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)

# ตรวจสอบและตั้งค่าอุปกรณ์ (GPU)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"อุปกรณ์ที่ใช้: {DEVICE}")


DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



อุปกรณ์ที่ใช้: cuda


In [2]:
# %% 2. การเตรียมข้อมูล

print("กำลังโหลด Thai Fake News Detection Dataset...")
# โหลดชุดข้อมูลจาก Hugging Face Hub
raw_dataset = load_dataset("EXt1/Thai-True-Fake-News")

กำลังโหลด Thai Fake News Detection Dataset...


In [3]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Title', 'Verification_Status'],
        num_rows: 6004
    })
})

In [4]:
from datasets import Dataset, DatasetDict

# แปลง Dataset เป็น DataFrame ก่อน
df = raw_dataset['train'].to_pandas()

# เปลี่ยนชื่อคอลัมน์ใน DataFrame
df = df.rename(columns={'Title': 'text', 'Verification_Status': 'label'})

# อัปเดต raw_dataset โดยแปลงจาก DataFrame กลับเป็น Dataset

# สร้าง Dataset ใหม่จาก DataFrame ที่เปลี่ยนชื่อคอลัมน์แล้ว
updated_dataset = Dataset.from_pandas(df)

# สร้าง DatasetDict ใหม่
raw_dataset = DatasetDict({
    'train': updated_dataset
})

print("เปลี่ยนชื่อคอลัมน์สำเร็จ:")
print(f"DataFrame columns: {df.columns.tolist()}")
print(f"Dataset features: {raw_dataset['train'].features}")
print("\nตัวอย่างข้อมูลหลังเปลี่ยนชื่อ:")
print(df[['text', 'label']].head())

เปลี่ยนชื่อคอลัมน์สำเร็จ:
DataFrame columns: ['Unnamed: 0', 'text', 'label']
Dataset features: {'Unnamed: 0': Value(dtype='float64', id=None), 'text': Value(dtype='string', id=None), 'label': Value(dtype='string', id=None)}

ตัวอย่างข้อมูลหลังเปลี่ยนชื่อ:
                                                text     label
0   กรมพัฒนาธุรกิจการค้าอนุญาตใบทะเบียนพาณิชย์ราย...  ข่าวปลอม
1   กรมการจัดหางานส่งเสริมให้ชาวไทยมีรายได้ เฉลี่...  ข่าวปลอม
2   ตลาดหลักทรัพย์แห่งประเทศไทยเปิดลงทุนเพื่อหาค่...  ข่าวปลอม
3   ตลาดหลักทรัพย์ฯ เปิดพอร์ตหุ้นธนาคารกำไรสูง 48...  ข่าวปลอม
4   ผู้บริหารธ. ออมสินไลน์เชิญชวนกู้เงินหลักหมื่น...  ข่าวปลอม


In [5]:
df

,Unnamed: 0,text,label
0,3413.0,กรมพัฒนาธุรกิจการค้าอนุญาตใบทะเบียนพาณิชย์ราย...,ข่าวปลอม
1,7926.0,กรมการจัดหางานส่งเสริมให้ชาวไทยมีรายได้ เฉลี่...,ข่าวปลอม
2,3285.0,ตลาดหลักทรัพย์แห่งประเทศไทยเปิดลงทุนเพื่อหาค่...,ข่าวปลอม
3,1972.0,ตลาดหลักทรัพย์ฯ เปิดพอร์ตหุ้นธนาคารกำไรสูง 48...,ข่าวปลอม
4,7318.0,ผู้บริหารธ. ออมสินไลน์เชิญชวนกู้เงินหลักหมื่น...,ข่าวปลอม
...,...,...,...
5999,7550.0,แรงงานต่างด้าวที่ทำงาน ถึง 13 ก.พ. 66 ต่ออายุป...,ข่าวจริง
6000,5205.0,กทม.คุมเข้มสุ่มตรวจร้านค้าสถานบริการ ปิด/ห้ามขาย,ข่าวจริง
6001,5379.0,ครม. อนุมัติ 12 มาตรการ แก้ปัญหาฝุ่นพิษ PM 2.5,ข่าวจริง
6002,6404.0,สธ. พร้อมฉีดวัคซีนบูสเตอร์ให้ผู้ที่ฉีดซิโนแวค ...,ข่าวจริง


In [6]:
# แปลงเป็น Pandas DataFrame เพื่อให้จัดการง่าย
df = raw_dataset['train'].to_pandas()

# ทำความสะอาดข้อมูลเบื้องต้น
df = df.dropna(subset=['text', 'label'])
df = df.drop_duplicates(subset=['text'])

# จำลองคอลัมน์ news_title และ news_content (ในการใช้งานจริงให้ใช้คอลัมน์จากโจทย์)
# ในที่นี้เราจะสมมติว่า 20 ตัวอักษรแรกคือ title และที่เหลือคือ content
df['news_title'] = df['text'].str[:20]
df['news_content'] = df['text'].str[20:]

# เปลี่ยน label 'fake' -> 1, 'real' -> 0
df['label'] = df['label'].apply(lambda x: 1 if x == 'ข่าวปลอม' else 0)

print(f"โหลดและเตรียมข้อมูลเบื้องต้นสำเร็จ จำนวน {len(df)} รายการ")
print("ตัวอย่างข้อมูล:")
print(df[['news_title', 'news_content', 'label']].head())

# --- แบ่งข้อมูลเป็น Train และ Test set เพื่อจำลองสถานการณ์การแข่งขัน ---
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# สร้าง news_id สำหรับ test set
test_df['news_id'] = range(len(test_df))

print(f"\nขนาดของ Train set: {len(train_df)}")
print(f"ขนาดของ Test set: {len(test_df)}")

# --- แบ่งข้อมูล Train เป็น Training และ Validation set สำหรับการ Fine-tune ---
# การมี Validation set เป็นสิ่งสำคัญมากเพื่อติดตามประสิทธิภาพและป้องกัน Overfitting
train_subset_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42, stratify=train_df['label'])

# แปลง Pandas DataFrames กลับไปเป็น Dataset object ของ Hugging Face
train_dataset = Dataset.from_pandas(train_subset_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

print("\nตัวอย่างข้อมูลใน Validation set:")
print(val_dataset)


โหลดและเตรียมข้อมูลเบื้องต้นสำเร็จ จำนวน 5749 รายการ
ตัวอย่างข้อมูล:
             news_title                                       news_content  \
0   กรมพัฒนาธุรกิจการค้  าอนุญาตใบทะเบียนพาณิชย์รายบุคคล ประกอบธุรกิจเง...   
1   กรมการจัดหางานส่งเส  ริมให้ชาวไทยมีรายได้ เฉลี่ย 1,500 – 3,900 บาทต...   
2   ตลาดหลักทรัพย์แห่งป  ระเทศไทยเปิดลงทุนเพื่อหาค่าข้าว วันละ 1,000 บา...   
3   ตลาดหลักทรัพย์ฯ เปิ                     ดพอร์ตหุ้นธนาคารกำไรสูง 48-50%   
4   ผู้บริหารธ. ออมสินไ  ลน์เชิญชวนกู้เงินหลักหมื่นถึงหลักแสน ปลอดภัย 100%   

   label  
0      1  
1      1  
2      1  
3      1  
4      1  

ขนาดของ Train set: 4599
ขนาดของ Test set: 1150

ตัวอย่างข้อมูลใน Validation set:
Dataset({
    features: ['Unnamed: 0', 'text', 'label', 'news_title', 'news_content', '__index_level_0__'],
    num_rows: 460
})


In [7]:
train_subset_df

,Unnamed: 0,text,label,news_title,news_content
4109,9271.0,กทม. เตือนพื้นที่เสี่ยง 16 ชุมชนนอกแนวกั้นน้ำเ...,0,กทม. เตือนพื้นที่เสี,่ยง 16 ชุมชนนอกแนวกั้นน้ำเฝ้าระวังน้ำท่วม
4876,4068.0,"ครม. อนุมัติงบประมาณ 7,660 ล้านบาท โครงการประก...",0,ครม. อนุมัติงบประมาณ,"7,660 ล้านบาท โครงการประกันรายได้เกษตรกรชาวสว..."
336,2745.0,กรมการจัดหางานต้องการผู้ร่วมงานในระยะยาว ทำงา...,1,กรมการจัดหางานต้องก,ารผู้ร่วมงานในระยะยาว ทำงาน 7 วัน รายได้ 500 –...
2463,5266.0,ประเทศไทยแพร่ระบาดระดับ 4 แล้ว,1,ประเทศไทยแพร่ระบาดร,ะดับ 4 แล้ว
5674,5065.0,11 โรงพยาบาลในสังกัดกทม. ตรวจคัดกรองโรคทั่วไปฟ...,0,11 โรงพยาบาลในสังกัด,กทม. ตรวจคัดกรองโรคทั่วไปฟรี เริ่ม 20-24 ก.ค. ...
...,...,...,...,...,...
1021,4708.0,ผลิตภัณฑ์ Nuriv ช่วยลดระดับน้ำตาลในเลือด,1,ผลิตภัณฑ์ Nuriv ช่ว,ยลดระดับน้ำตาลในเลือด
2845,3215.0,กรมการจัดหางานเปิดรับสมัครแรงงานโรงงานไฟฟ้าที...,1,กรมการจัดหางานเปิดร,ับสมัครแรงงานโรงงานไฟฟ้าที่เกาหลี
239,1784.0,เบอร์ 1175 และ .AIS1175 หากรับสายจะถูกดูดเงิน...,1,เบอร์ 1175 และ .AIS,1175 หากรับสายจะถูกดูดเงินออกจากบัญชี
3985,3935.0,Easy pass สามารถใช้ผ่านทางยกระดับดอนเมืองโทลล์...,0,Easy pass สามารถใช้ผ,่านทางยกระดับดอนเมืองโทลล์เวย์ได้ ตั้งแต่วันที...


In [8]:
# %% 3. การ Tokenization

MODEL_NAME = "airesearch/wangchanberta-base-att-spm-uncased"
print(f"กำลังโหลด Tokenizer จาก '{MODEL_NAME}'...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    """
    ฟังก์ชันสำหรับ Tokenize โดยการนำ title และ content มาต่อกัน
    """
    # สร้างข้อความที่รวมกัน: "title [SEP] content"
    combined_text = [
        title + tokenizer.sep_token + content
        for title, content in zip(examples["news_title"], examples["news_content"])
    ]
    # ทำการ tokenize ข้อความ
    return tokenizer(combined_text, padding="max_length", truncation=True, max_length=256)

# ใช้ .map() เพื่อทำการ tokenize ข้อมูลทั้งหมดอย่างรวดเร็ว
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

# ลบคอลัมน์ที่ไม่จำเป็นออก เหลือไว้แต่ที่ Trainer ต้องใช้
tokenized_train_dataset = tokenized_train_dataset.remove_columns(['news_title', 'news_content', 'text', '__index_level_0__'])
tokenized_val_dataset = tokenized_val_dataset.remove_columns(['news_title', 'news_content', 'text', '__index_level_0__'])
tokenized_test_dataset = tokenized_test_dataset.remove_columns(['news_title', 'news_content', 'text', 'label', 'news_id', '__index_level_0__'])


print("\nTokenize ข้อมูลสำเร็จ")
print("ตัวอย่างข้อมูลที่ผ่านการ Tokenize แล้ว:")
print(tokenized_train_dataset[0])


กำลังโหลด Tokenizer จาก 'airesearch/wangchanberta-base-att-spm-uncased'...


Map:   0%|          | 0/4139 [00:00<?, ? examples/s]

Map:   0%|          | 0/460 [00:00<?, ? examples/s]

Map:   0%|          | 0/1150 [00:00<?, ? examples/s]


Tokenize ข้อมูลสำเร็จ
ตัวอย่างข้อมูลที่ผ่านการ Tokenize แล้ว:
{'Unnamed: 0': 9271.0, 'label': 0, 'input_ids': [5, 10, 1275, 11, 10, 1328, 214, 150, 270, 6, 10, 369, 4115, 10, 531, 10, 760, 1075, 1004, 4474, 143, 3842, 1350, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [19]:
tokenized_val_dataset

Dataset({
    features: ['Unnamed: 0', 'label', 'input_ids', 'attention_mask'],
    num_rows: 460
})

In [9]:
# %% 4. การโหลดโมเดลและกำหนดค่า

print(f"กำลังโหลดโมเดล '{MODEL_NAME}' สำหรับ Sequence Classification...")
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2  # 2 labels: 0 (reliable) and 1 (misinformation)
).to(DEVICE)

# กำหนดฟังก์ชันสำหรับคำนวณค่า метริก (Metrics) ระหว่างการฝึก
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1}

print("โหลดโมเดลและตั้งค่าฟังก์ชัน Metrics สำเร็จ")


กำลังโหลดโมเดล 'airesearch/wangchanberta-base-att-spm-uncased' สำหรับ Sequence Classification...


Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


โหลดโมเดลและตั้งค่าฟังก์ชัน Metrics สำเร็จ


In [ ]:
# %% 5. การ Fine-tune โมเดลด้วย Trainer API

# สร้างโฟลเดอร์ results ในตำแหน่งที่ไม่มีอักขระพิเศษ
import os
import tempfile

# วิธีที่ 1: ใช้ temporary directory
output_dir = tempfile.mkdtemp(prefix="nlp_results_")

# วิธีที่ 2: หรือใช้ path ที่เรียบง่าย (ถ้าวิธีที่ 1 ไม่ทำงาน)
# output_dir = r"C:\temp\nlp_results"
# os.makedirs(output_dir, exist_ok=True)

print(f"Output directory: {output_dir}")

# กำหนด Training Arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,                 # จำนวนรอบในการฝึก
    per_device_train_batch_size=16,     # ขนาด batch สำหรับ train
    per_device_eval_batch_size=16,      # ขนาด batch สำหรับ eval
    warmup_steps=500,                   # จำนวน step สำหรับ warm up learning rate
    weight_decay=0.01,                  # ค่า weight decay
    logging_steps=100,
    evaluation_strategy="epoch",        # ประเมินผลทุกๆ 1 epoch
    save_strategy="epoch",              # บันทึกโมเดลทุกๆ 1 epoch
    load_best_model_at_end=True,        # โหลดโมเดลที่ดีที่สุดหลังฝึกเสร็จ
    metric_for_best_model="f1",         # ใช้ f1 score เป็นเกณฑ์เลือกโมเดลที่ดีที่สุด
)

# สร้าง Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

print(f"กำลังเริ่มการ Fine-tune โมเดล... (บันทึกผลลัพธ์ที่: {output_dir})")
# เริ่มการฝึกโมเดล
trainer.train()

print("\nการ Fine-tune เสร็จสิ้น")

Output directory: C:\Users\potij\AppData\Local\Temp\nlp_results_y5td05jg
กำลังเริ่มการ Fine-tune โมเดล... (บันทึกผลลัพธ์ที่: C:\Users\potij\AppData\Local\Temp\nlp_results_y5td05jg)


C:\Users\potij\AppData\Local\Temp\ipykernel_23636\1148209924.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit 

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.443300,0.368592,0.847826,0.853556
2,0.311300,0.346188,0.884783,0.875294
3,0.187500,0.310403,0.908696,0.904545



การ Fine-tune เสร็จสิ้น


Exception in thread IntMsgThr:
Traceback (most recent call last):
  File "c:\Users\potij\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1052, in _bootstrap_inner
    self.run()
  File "C:\Users\potij\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\potij\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 989, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\potij\AppData\Local\Programs\Python\Python312\Lib\site-packages\wandb\sdk\wandb_run.py", line 315, in check_internal_messages
    self._loop_check_status(
  File "c:\Users\potij\AppData\Local\Programs\Python\Python312\Lib\site-packages\wandb\sdk\wandb_run.py", line 225, in _loop_check_status
    local_handle = request()
                   ^^^^^^^^^
  File "c:\Users\potij\AppData\Local\Programs\Python\Python312\Lib\site-packages\wandb\sdk\interface\interface.py", line 938, in deliver_inter

In [17]:
# %% 6. การทำนายผลสำหรับ Test Set

print("กำลังทำนายผลบน Test Set...")

# ใช้ trainer.predict เพื่อรับผลลัพธ์
predictions = trainer.predict(tokenized_test_dataset)

# ค่าที่ได้จะเป็น logits, เราต้องหา argmax เพื่อให้ได้ label ที่มีความเป็นไปได้สูงสุด
predicted_labels = np.argmax(predictions.predictions, axis=1)

print("ทำนายผลสำเร็จ")
print("ตัวอย่างผลการทำนาย (labels):", predicted_labels[:20])

# (ทางเลือก) ประเมินผลกับคำตอบจริงของ test set เพื่อดูประสิทธิภาพ
actual_labels = test_df['label'].values
accuracy = accuracy_score(actual_labels, predicted_labels)
f1 = f1_score(actual_labels, predicted_labels)

print(f"\nประสิทธิภาพบน Test Set (เพื่อการประเมิน):")
print(f"  - Accuracy: {accuracy:.4f}")
print(f"  - F1-Score: {f1:.4f}")


กำลังทำนายผลบน Test Set...


ทำนายผลสำเร็จ
ตัวอย่างผลการทำนาย (labels): [1 1 0 1 1 1 0 0 1 1 0 0 1 0 1 1 0 0 1 0]

ประสิทธิภาพบน Test Set (เพื่อการประเมิน):
  - Accuracy: 0.9043
  - F1-Score: 0.9000


In [20]:
# %% 7. การสร้างไฟล์ Submission

print("กำลังสร้างไฟล์ submission.csv...")

# สร้าง DataFrame สำหรับไฟล์ submission
submission_df = pd.DataFrame({
    'news_id': test_df['news_id'],
    'label': predicted_labels
})

print("\nตัวอย่างข้อมูลในไฟล์ Submission:")
print(submission_df.head())

# บันทึกเป็นไฟล์ CSV โดยไม่มี index
submission_df.to_csv("submission.csv", index=False)

print("\nสร้างไฟล์ submission.csv สำเร็จ!")


กำลังสร้างไฟล์ submission.csv...

ตัวอย่างข้อมูลในไฟล์ Submission:
      news_id  label
4666        0      1
1512        1      1
4304        2      0
2085        3      1
1592        4      1

สร้างไฟล์ submission.csv สำเร็จ!
